The offensive model adapted from Berkem (past MSc student working on related work).

In [ ]:
!pip install datasets
!pip install pycountry
!pip install pandas==1.2.3
!pip install transformers
!pip install torch

In [ ]:
import pandas as pd
import numpy as np
import transformers
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

#irony
import urllib.request
from scipy.special import softmax
import csv

# offensiveness
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier


In [ ]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Ziwen 


In [ ]:
def run_offensive_model(test):
    df_scraped = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/3/offensiveness/labeled_tweets.csv')
    df_public = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/3/offensiveness/public_data_labeled.csv')
    df_scraped.drop_duplicates(inplace = True)
    df_scraped.drop('id', axis = 'columns', inplace = True)
    df_public.drop_duplicates(inplace = True)
    df = pd.concat([df_scraped, df_public])
    df['label'] = df.label.map({'Offensive': 1, 'Non-offensive': 0})
    X_train, X_test, y_train, y_test = train_test_split(df['full_text'], 
                                                df['label'], 
                                                random_state=42)


    count_vector = CountVectorizer(stop_words = 'english', lowercase = True)

    training_data = count_vector.fit_transform(X_train)
    testing_data = count_vector.transform(test)
    model = SGDClassifier()
    model.fit(training_data, y_train)
    preds = model.predict(testing_data)
    return preds


def load_offensive_model(df):
    test_data = df.text
    preds = run_offensive_model(test_data)
    df['offensive'] = preds
    return df


def load_irony_model(df):
    device = torch.device("cuda")
    task='irony'
    MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    tokens = df.text.apply(lambda row: tokenizer(row, return_tensors='pt'))
    # download label mapping
    mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
    with urllib.request.urlopen(mapping_link) as f:
        html = f.read().decode('utf-8').split("\n")
        csvreader = csv.reader(html, delimiter='\t')
    irony_labels = [row[1] for row in csvreader if len(row) > 1]
    # PT
    with torch.no_grad():
        global irony_model 
        irony_model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)
    df['cardiff_tokens'] = tokens
    print('loaded irony model for tweets')


In [ ]:
def get_irony(df,index,row):
    output = irony_model(**row.cardiff_tokens.to(device))
    scores = output[0][0].detach().cpu().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    df.loc[index, 'irony'] = ranking[0]

    

def get_offensive(df,index,row):
    df.loc[index, 'offensive'] = row.offensive

In [ ]:
hashtags = ['#VoteThemOut',
            # '#ToryScumOut',
            # 'F1',
            'Lisa',
            '#EnoughIsEnough',
            '#EnergyPrices',
            '#iOS16',
            '#taiwan',
            '#onepiece',
            '#CostOfLivingCrisis',
            '#GetBackToWorkYouFatPonce', 
            # '#ClosingCeremony',
            '#BookLoversDay',
            '#biden']
            
# add this 
# hashtags = ['F1']

for hash in hashtags:
        device = torch.device("cuda")
        df = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/2/{hash}_tweet_features_with_politeness.csv') 
        #id,	text,	created_at,	user_id,	followers_count,	friends_count	,favourites_count,	retweet_count


        loaders = [load_irony_model, load_offensive_model]

        [load(df) for load in loaders]

        score_funcs = [get_offensive, get_irony]

        [ func(df,index,row) for func in score_funcs for index, row in df.iterrows() ]

        df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/3/{hash}_tweet_features_with_offensiveirony.csv')
 